In [1]:
import pandas as pd
import time
import requests, bs4

Quick scrappers for headlines from https://www.theonion.com/ and https://www.huffpost.com. Structure of these websites can change often, so this might not work in the future.

Scrapping was performed 19.10.2022.

### The Onion

In [2]:
start_url = 'https://www.theonion.com/latest'
res = requests.get(start_url)
soup = bs4.BeautifulSoup(res.text, 'html.parser')
headlines = []

In [3]:
headlines += [headline.get_text() for headline in soup.select('a > h2')]
next_url = start_url + soup.select('div[class="row"]> div > a')[0].get('href')

In [4]:
for i in range(25):
    res = requests.get(next_url)
    soup = bs4.BeautifulSoup(res.text, 'html.parser')
    headlines += [headline.get_text() for headline in soup.select('a > h2')]
    next_url = start_url + soup.select('div[class="row"]> div > a')[0].get('href')
    time.sleep(0.2)

In [5]:
onion_df = pd.DataFrame({'headline':headlines, 'is_sarcastic': len(headlines) * [1], 'website': len(headlines) * ['the Onion']})

In [6]:
onion_df

,headline,is_sarcastic,website
0,Late Night Host James Corden Briefly Banned Fr...,1,the Onion
1,Janet Yellen Rolls Up Sleeves To Take Another ...,1,the Onion
2,Moon Dead At 29,1,the Onion
3,Former High School Basketball Star Lands AA Sp...,1,the Onion
4,Pros And Cons Of E-Bikes,1,the Onion
...,...,...,...
515,Landlord Informs Tenants Of Upcoming Improveme...,1,the Onion
516,Sacklers Ask Friend For Hookup To Buy Pharmace...,1,the Onion
517,Mom Compliments Foreigner On How Cheap Everyth...,1,the Onion
518,Dennis Rodman Plans Trip To Russia To Seek Bri...,1,the Onion


### Huffington Post

In [11]:
headlines = []
for page in range(50):
    url = f'https://www.huffpost.com/news/?page={page+1}'
    page = requests.get(url).text
    soup = bs4.BeautifulSoup(page, 'html.parser')
    headlines += [headline.get_text() for headline in soup.select('h3')]
    time.sleep(0.2)

huff_df = pd.DataFrame({'headline': headlines})
huff_df.drop_duplicates(inplace=True, ignore_index=True)
huff_df['is_sarcastic'] = 0
huff_df['website'] = 'huffpost.com'

In [12]:
huff_df

,headline,is sarcastic,website
0,Worker Admits Dumping Raw Waste Into Jackson W...,0,huffpost.com
1,Jordan Klepper Has Baffling Chat With Michigan...,0,huffpost.com
2,Bubba Wallace Suspended By NASCAR For 'Dangero...,0,huffpost.com
3,Republican Group Names And Shames GOP’s ‘Big L...,0,huffpost.com
4,Pelosi Taunts Trump For Lack Of Courage On Jan...,0,huffpost.com
...,...,...,...
510,Feds Vow Major Aid For Hurricane Ian Victims A...,0,huffpost.com
511,Texas Man Charged With Migrant's Death Ran Det...,0,huffpost.com
512,"Bolsonaro, Lula Neck-And-Neck In Polarized Bra...",0,huffpost.com
513,9 NATO Countries Throw Support Behind Ukraine ...,0,huffpost.com


In [20]:
huff_df.rename(columns={'is sarcastic': 'is_sarcastic'}, inplace=True)
huff_df

,headline,is_sarcastic,website
0,Worker Admits Dumping Raw Waste Into Jackson W...,0,huffpost.com
1,Jordan Klepper Has Baffling Chat With Michigan...,0,huffpost.com
2,Bubba Wallace Suspended By NASCAR For 'Dangero...,0,huffpost.com
3,Republican Group Names And Shames GOP’s ‘Big L...,0,huffpost.com
4,Pelosi Taunts Trump For Lack Of Courage On Jan...,0,huffpost.com
...,...,...,...
510,Feds Vow Major Aid For Hurricane Ian Victims A...,0,huffpost.com
511,Texas Man Charged With Migrant's Death Ran Det...,0,huffpost.com
512,"Bolsonaro, Lula Neck-And-Neck In Polarized Bra...",0,huffpost.com
513,9 NATO Countries Throw Support Behind Ukraine ...,0,huffpost.com


### Concatenating and saving to csv

In [21]:
df = pd.concat([onion_df, huff_df])
df = df.sample(frac=1, ignore_index=True)

In [22]:
df

,headline,is_sarcastic,website
0,Most Glaring Times Trump's Children Have Broke...,1,the Onion
1,Anna Faris Names Director She Says Touched Her...,0,huffpost.com
2,Guy In Line For Port-A-Potty Won’t Stop Assuri...,1,the Onion
3,Embattled GOP Candidate J.R. Majewski Also Exa...,0,huffpost.com
4,Rep. Lofgren Teases 'Pretty Surprising' Materi...,0,huffpost.com
...,...,...,...
1030,Watch A Honey Badger Fight 3 Leopards In 'Unbe...,0,huffpost.com
1031,Biden Secures Liberal Priorities With Little R...,0,huffpost.com
1032,Pilot Sues Southwest After Colleague Exposed H...,0,huffpost.com
1033,Annoying Boyfriend Always Leaves Toilet Seat R...,1,the Onion


In [23]:
df.to_csv('fresh_headlines.csv')